# Interrupción Legal del Embarazo

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' ##### API SÓLO TRAE INFO 2016
FILE_NAME = 'interrupcion_embarazo'

## Import

In [2]:
from mariachis.models import InterrupcionEmbarazo
ile = InterrupcionEmbarazo(BASE_DIR, FILE_NAME)
print(ile)

Directorio: 	/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


In [3]:
##### API SÓLO TRAE INFO 2016
ile.full_import(api=True, api_export=False, resource_id=RESOURCE_ID)['ano'].value_counts(1)

Archivo importado desde: https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=932b56bf-c5ec-4815-9814-370d58754002&distinct=true&limit=32000
Con 1000 renglones y 48 columnas


2016    1.0
Name: ano, dtype: float64

In [4]:
df = ile.full_import(api=False)
df.sample()

Archivo con nombre interrupcion_embarazo.csv fue encontrado en /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data
Con 79403 renglones y 47 columnas


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,tanalgesico,cconsejo,panticoncep,fecha_cierre,resultado_ile,medicamento,clues_2,clues_hospital,procile_simplificada,procile_completa
60835,2018.0,MAYO,2018-05-09,NaN,UNION LIBRE,43.0,NINGUNO,PRIMARIA,NaN,OTRA,...,NaN,NaN,NaN,NaN,COMPLETA,NaN,DFSSA003932,DFSSA003932,MEDICAMENTO,MISOPROSTOL + MIFEPRISTONA


## Diccionario de variables

### Limpiar

In [5]:
# Diccionario para aplicar limpieza por tipo de variable
clean_dict = {}

# Variables que o son 0 o > 0
clean_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']

# Numéricas que serán partidas en rangos
clean_dict['vars_num'] = {
    'edad': [9,11,15,18,21,25,29,35],
    'menarca': [7,11,12,13,14,15],
    'fsexual': [7,11,15,18,21],
    'sememb': [3,6,9,12,20],
    'nhijos': [0,1,2],
    'gesta': [0,1,2,3,4],
    'c_num': [0,1,2],
    'p_semgest': [0,5,6,7,8,9],
    'p_diasgesta': [0,1,2,3,4,5]
}

# Obtener la primer palabra
clean_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Binarias de sí o no
clean_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile']

# Limpiar y mantener para clustering posterior
clean_dict['vars_clean_keep'] = ['motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep', 'procile_simplificada', 'procile_completa']

# Sólo limpiar
clean_dict['vars_just_clean'] = ['entidad','alc_o_municipio']

# Agrupar categorías
clean_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',
        'SEPARADA':'SEPARADX',
        'VIUDA':'SEPARADX',
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'DESEMPLEADX',
    },
    'desc_derechohab':{
        'NINGUNO':'NINGUNO',
        'IMSS':'ALGUNO',
        'SEGURO POPULAR':'ALGUNO',
        'OTRA':'ALGUNO',
        'ISSSTE':'ALGUNO',
    },
    'nivel_edu':{
        'LICENCIATURA':'LICENCIATURA',
        'MAESTRIA':'POSGRADO',
        'DOCTORADO':'POSGRADO',
        'PREPARATORIA':'PREPARATORIA',
        'SECUNDARIA':'SECUNDARIA',
        'PRIMARIA':'PRIMARIA',
        'SIN ACCESO A LA EDUCACION FORMAL':'SIN_ACCESO',
    },
    'parentesco':{
        'PAREJA':'PAREJA',
        'EX-PAREJA':'PAREJA',
        'FAMILIAR MUJER':'FAMILIAR',
        'FAMILIAR HOMBRE':'FAMILIAR',
        'FAMILIAR SIN ESPECIFICAR':'FAMILIAR',
        'TUTOR O RESPONSABLE LEGAL':'FAMILIAR',
        'PERSONA CERCANA':'AMIGX',
        'PERSONA CERCANA MUJER':'AMIGX',
        'PERSONA CERCANA HOMBRE':'AMIGX',
        'AMIGA':'AMIGX',
        'AMIGO':'AMIGX',
        'CONOCIDA':'OTRX',
        'CONOCIDO':'OTRX',
        'RELACION LABORAL':'OTRX',
        'OTRA':'OTRX',
    },
    'religion':{
        'NINGUNA':'NINGUNA',
        'CATOLICA':'SI',
        'CRISTIANA':'SI',
        'OTRA':'SI',
        'MORMONA':'SI',
        'TESTIGA DE JEHOVA':'SI',
        'BUDISTA':'SI',
        'JUDIA':'SI',
        'ANGLICANA':'SI',
        'MUSULMANA':'SI',
    },
}

#### Auxiliar para rangos de variables numéricas

In [6]:
from pandas import qcut
from IPython.display import display

aux_dict = {}
for col in clean_dict['vars_num']:
    new_col = df[col].map(ile.clean_number).astype(float)
    orig_bins = qcut(new_col, q=6, retbins=True, duplicates='drop')[-1]
    aux_dict[col] = list(orig_bins[:-1])
display(aux_dict)

{'edad': [11.0, 20.0, 22.0, 25.0, 28.0, 32.0],
 'menarca': [0.0, 11.0, 12.0, 13.0, 14.0],
 'fsexual': [0.0, 15.0, 16.0, 17.0, 18.0, 19.0],
 'sememb': [0.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'nhijos': [0.0, 1.0, 2.0],
 'gesta': [0.0, 1.0, 2.0, 3.0, 4.0],
 'c_num': [0.0, 1.0, 2.0],
 'p_semgest': [0.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'p_diasgesta': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]}

### Crear variables

In [7]:
# Diccionario para crear variables
vars_dict = {}

# Para calcular la diferencia entre edades
vars_dict['edad_1a_menst'] = 'menarca'
vars_dict['edad_vida_sex'] = 'fsexual'
vars_dict['edad_actual'] = 'edad'

# Para comparar los métodos anticonceptivos
vars_dict['metodo_antes'] = 'anticonceptivo'
vars_dict['metodo_despues'] = 'panticoncep'

## Transformar

In [8]:
df, cluster_cols = ile.wrangling_ile(df, clean_dict, vars_dict, export_result=True)

Archivo: interrupcion_embarazo_clean.csv fue exportado exitosamente en: /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


In [9]:
ile.cool_print(f'\nLas {len(cluster_cols)} columnas para clustering son:\n{cluster_cols}\n\n', sleep_time=0.01)
df.sample()


Las 38 columnas para clustering son:
['procile_simplificada', 'cconsejo', 'procile_completa', 'p_consent', 'resultado_ile', 'anticonceptivo', 'autoref', 'motiles', 'desc_derechohab', 'religion', 'ocupacion', 'desc_servicio', 'parentesco', 'tanalgesico', 'panticoncep', 'c_dolor', 'nivel_edu', 'consejeria', 's_complica', 'edocivil_descripcion', 'rango_edad', 'rango_menarca', 'rango_fsexual', 'rango_sememb', 'rango_nhijos', 'rango_gesta', 'rango_c_num', 'rango_p_semgest', 'rango_p_diasgesta', 'naborto', 'npartos', 'ncesarea', 'nile', 'fsexual_vs_menarca', 'edad_vs_menarca', 'edad_vs_fsexual', 'antes_vs_despues', 'antes_vs_despues_detalle']




,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,rango_nhijos,rango_gesta,rango_c_num,rango_p_semgest,rango_p_diasgesta,rango_fsexual_vs_menarca,rango_edad_vs_menarca,rango_edad_vs_fsexual,antes_vs_despues,antes_vs_despues_detalle
64753,2018.0,OCTUBRE,2018-10-04,DESCONOCIDO,UNION_LIBRE,21.0,NINGUNO,LICENCIATURA,ESTUDIANTE,SI,...,00,01,02,07,05,04,01 a 08,01 a 04,DIFERENTE,"antes: CONDON, después: PASTILLA"


## Clustering

In [10]:
df, clustering_pipeline = ile.clustering_ile(df, cluster_cols, n_clusters=7)
df.sample()

Archivo: interrupcion_embarazo_cluster.csv fue exportado exitosamente en: /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,rango_gesta,rango_c_num,rango_p_semgest,rango_p_diasgesta,rango_fsexual_vs_menarca,rango_edad_vs_menarca,rango_edad_vs_fsexual,antes_vs_despues,antes_vs_despues_detalle,cluster
73545,2018.0,MAYO,2018-05-15,DESCONOCIDO,SOLTERX,33.0,NINGUNO,SECUNDARIA,DESCONOCIDO,NINGUNA,...,01,01,01 a 05,DESCONOCIDO,05 a 06,>= 18,>= 13,DIFERENTE,"antes: NINGUNO, después: DESCONOCIDO",F


## Checkpoint

In [11]:
## QUITAR COMENTARIOS SI YA SE HIZO EL CLUSTERING

# from pathlib import Path
# from pandas import read_csv

# BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# # BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
# RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' ##### API SÓLO TRAE INFO 2016
# FILE_NAME = 'interrupcion_embarazo'

# from mariachis.models import InterrupcionEmbarazo
# ile = InterrupcionEmbarazo(BASE_DIR, FILE_NAME)

# cluster_cols = ['panticoncep', 'desc_derechohab', 'p_consent', 'anticonceptivo', 'resultado_ile', 'procile_simplificada', 'religion', 'nivel_edu', 'procile_completa', 'desc_servicio', 'motiles', 'autoref', 'edocivil_descripcion', 'ocupacion', 'parentesco', 's_complica', 'consejeria', 'cconsejo', 'tanalgesico', 'c_dolor', 'rango_edad', 'rango_menarca', 'rango_fsexual', 'rango_sememb', 'rango_nhijos', 'rango_gesta', 'rango_c_num', 'rango_p_semgest', 'rango_p_diasgesta', 'naborto', 'npartos', 'ncesarea', 'nile', 'fsexual_vs_menarca', 'edad_vs_menarca', 'edad_vs_fsexual', 'antes_vs_despues', 'antes_vs_despues_detalle']

# df = read_csv(Path(BASE_DIR).joinpath(f'{FILE_NAME}_cluster.csv'), low_memory=False)
# df.sample()

## Perfilamiento

### Recuento

In [12]:
add_cols = ['fingreso_yearquarter','entidad']
ile.profiles(df[add_cols+cluster_cols+['cluster']])

In [15]:
df['cluster'].value_counts(1, dropna=False)

E    0.233984
F    0.173432
B    0.141519
D    0.137816
C    0.131254
A    0.115978
G    0.066018
Name: cluster, dtype: float64

### Interpretación

In [13]:
cluster_caract = {
    'fingreso_yearquarter':{
        'A':'Fuerte en 2020',
        'B':'Fuerte en 2019 y registros sin fecha',
        'C':'Fuerte en 2016',
        'D':'Fuerte en 2017-2018',
        'E':'Fuerte en 2019',
        'F':'',
        'G':'',
    },
    'entidad':{
        'A':'',
        'B':'Foránexs',
        'C':'',
        'D':'Capitalinxs',
        'E':'Mexiquensxs',
        'F':'',
        'G':'',
    },
    'resultado_ile':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'tanalgesico':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'parentesco':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'religion':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'edocivil_descripcion':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'nivel_edu':{
        'A':'Posgrado',
        'B':'No se sabe la educación',
        'C':'Secundaria, primaria y sin acceso a la educación',
        'D':'Preparatoria y licenciatura',
        'E':'',
        'F':'',
        'G':'',
    },
    'motiles':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'autoref':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'ocupacion':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'procile_simplificada':{
        'A':'',
        'B':'ASPIRACION ENDOUTERINA MANUAL O ELECTRICA',
        'C':'Medicamento',
        'D':'O no se sabe el procedimiento o medicamento',
        'E':'',
        'F':'',
        'G':'',
    },
    'procile_completa':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    's_complica':{
        'A':'No hubo complicaciones',
        'B':'Sí hubo comlicaciones',
        'C':'',
        'D':'No se sabe si hubo complicaciones',
        'E':'',
        'F':'',
        'G':'',
    },
    'cconsejo':{
        'A':'Sí hubo consejo',
        'B':'Sí y no hubo consejo',
        'C':'No se sabe si hubo consejo',
        'D':'No se sabe si hubo consejo',
        'E':'No hubo consejo',
        'F':'',
        'G':'',
    },
    'p_consent':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'desc_servicio':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'consejeria':{
        'A':'',
        'B':'No se sabe si hubo consejería',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'c_dolor':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'desc_derechohab':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'anticonceptivo':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'panticoncep':{
        'A':'Posterior: Implante',
        'B':'Posterior: Condón, ligadura',
        'C':'Posterior: DIU, inyección',
        'D':'Posterior: Parche, pastilla o no se conoce',
        'E':'',
        'F':'',
        'G':'',
    },
    'rango_edad':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'rango_menarca':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'rango_fsexual':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'rango_sememb':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'rango_nhijos':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'rango_gesta':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'rango_c_num':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'rango_p_semgest':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'rango_p_diasgesta':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'naborto':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'npartos':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'ncesarea':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'nile':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'fsexual_vs_menarca':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'edad_vs_menarca':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'edad_vs_fsexual':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'antes_vs_despues':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'antes_vs_despues_detalle':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
    'n':{
        'A':'',
        'B':'',
        'C':'',
        'D':'',
        'E':'',
        'F':'',
        'G':'',
    },
}

### Resumen

In [14]:
from pandas import DataFrame

DataFrame(cluster_caract).T.to_dict()

{'A': {'fingreso_yearquarter': 'Fuerte en 2020',
  'entidad': '',
  'resultado_ile': '',
  'tanalgesico': '',
  'parentesco': '',
  'religion': '',
  'edocivil_descripcion': '',
  'nivel_edu': 'Posgrado',
  'motiles': '',
  'autoref': '',
  'ocupacion': '',
  'procile_simplificada': '',
  'procile_completa': '',
  's_complica': 'No hubo complicaciones',
  'cconsejo': 'Sí hubo consejo',
  'p_consent': '',
  'desc_servicio': '',
  'consejeria': '',
  'c_dolor': '',
  'desc_derechohab': '',
  'anticonceptivo': '',
  'panticoncep': 'Implante',
  'rango_edad': '',
  'rango_menarca': '',
  'rango_fsexual': '',
  'rango_sememb': '',
  'rango_nhijos': '',
  'rango_gesta': '',
  'rango_c_num': '',
  'rango_p_semgest': '',
  'rango_p_diasgesta': '',
  'naborto': '',
  'npartos': '',
  'ncesarea': '',
  'nile': '',
  'fsexual_vs_menarca': '',
  'edad_vs_menarca': '',
  'edad_vs_fsexual': '',
  'antes_vs_despues': '',
  'antes_vs_despues_detalle': '',
  'n': ''},
 'B': {'fingreso_yearquarter': 'Fu